In [11]:
!pip install tensorflow numpy nltk
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [12]:
from nltk.corpus import gutenberg
import numpy as np
import random
import tensorflow as tf

# Choose a text (e.g., Shakespeare’s "Hamlet")
text = gutenberg.raw('shakespeare-hamlet.txt').lower()

# Tokenization: Convert text into characters
chars = sorted(list(set(text)))
char_to_int = {char: number for number, char in enumerate(chars)}
int_to_char = {number: char for number, char in enumerate(chars)}

# Create sequences of fixed length (e.g., 50 characters)
seq_length = 50
sequences = []
next_chars = []
for i in range(len(text) - seq_length):
    sequences.append(text[i:i + seq_length])
    next_chars.append(text[i + seq_length])

# Convert sequences to integer indices
X = np.zeros((len(sequences), seq_length, len(chars)), dtype=np.bool_)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool_)
for i, seq in enumerate(sequences):
    for j, char in enumerate(seq):
        X[i, j, char_to_int[char]] = 1
    y[i, char_to_int[next_chars[i]]] = 1

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(chars), activation='softmax')
])

# Change 'lr' to 'learning_rate'
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [15]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('text_generation_lstm.h5', save_best_only=True)
model.fit(X, y, batch_size=128, epochs=3, callbacks=[checkpoint])

Epoch 1/3
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 400s 314ms/step - loss: 2.7586
Epoch 2/3


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


1273/1273 ━━━━━━━━━━━━━━━━━━━━ 443s 315ms/step - loss: 2.1116
Epoch 3/3
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 444s 317ms/step - loss: 1.9382


In [31]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer  # Import the Tokenizer
from tensorflow.keras.utils import to_categorical #Import to_categorical for one-hot encoding

def generate_text(model, tokenizer, seed_text, max_sequence_length, temperature=1.0, num_chars=100):
    """
    Generates text from a trained model.

    Parameters:
        model: Trained LSTM model
        tokenizer: Tokenizer used during training (to convert text to tokens)
        seed_text: Initial text to start the generation
        max_sequence_length: Maximum sequence length used in training
        temperature: Sampling temperature (lower is more deterministic, higher is more random)
        num_chars: Number of characters to generate

    Returns:
        Generated text
    """
    generated_text = seed_text
    # Tokenize and pad the seed text
    input_sequence = tokenizer.texts_to_sequences([seed_text])[0]
    input_sequence = np.pad(input_sequence, (max_sequence_length - len(input_sequence), 0), mode='constant')

    # Reshape and one-hot encode the input sequence using the correct vocabulary size
    input_sequence = input_sequence.reshape(1, max_sequence_length)
    input_sequence = to_categorical(input_sequence, num_classes=len(tokenizer.word_index) + 1) # Using correct vocabulary size

    # Generate characters one by one
    for _ in range(num_chars):
        prediction_probs = model.predict(input_sequence)

        # Apply temperature to prediction probabilities
        prediction_probs = prediction_probs[:, -1, :]  # Focus on the last predicted character
        prediction_probs = np.asarray(prediction_probs).flatten()
        prediction_probs = np.log(prediction_probs + 1e-7) / temperature
        prediction_probs = np.exp(prediction_probs) / np.sum(np.exp(prediction_probs))

        # Sample a character from the predicted probabilities, ensuring it's within the vocabulary
        predicted_index = np.random.choice(range(len(prediction_probs)), p=prediction_probs)

        # Check if predicted_index is within the vocabulary
        if predicted_index in tokenizer.index_word:
            predicted_char = tokenizer.index_word[predicted_index]
        else:
            # Handle the case where the predicted index is out of vocabulary
            # You could use a special token like "<UNK>" or choose the most frequent character
            predicted_char = tokenizer.index_word[1]  # Example: using the most frequent character

        # Append predicted character to the generated text
        generated_text += predicted_char

        # Update the input sequence with the new character
        input_sequence = np.roll(input_sequence, shift=-1, axis=1)
        input_sequence[0, -1] = to_categorical(predicted_index, num_classes=len(tokenizer.word_index) + 1)

    return generated_text

In [21]:
history = model.history.history

In [23]:
import pickle

# Save the training history
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history, f)

In [ ]:
print(history.keys())

In [ ]:
import matplotlib.pyplot as plt

# Plot the training and validation loss over epochs
plt.figure(figsize=(8, 5))
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
plt.show()
# Print the final loss values
final_train_loss = history['loss'][-1]
final_val_loss = history['val_loss'][-1]

print(f"Final Training Loss: {final_train_loss:.4f}")
print(f"Final Validation Loss: {final_val_loss:.4f}")